In [1]:
pip install torch torchvision timm albumentations scikit-learn matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --upgrade numpy scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 57.0 kB/s eta 0:00:000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 77.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 102.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.3.5 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.

In [3]:
!pip install -U albumentations


  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.5
    Uninstalling numpy-2.3.5:
      Successfully uninstalled numpy-2.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
datasets 

In [ ]:
import os
os.kill(os.getpid(), 9)

In [7]:
# -----------------------------
# Imports
# -----------------------------
import os, glob
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
import timm
from tqdm import tqdm
import random

# -----------------------------
# Dataset
# -----------------------------
class UTKFaceDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")
        age = int(os.path.basename(img_path).split('_')[0])

        img = np.array(img)
        if self.transform:
            img = self.transform(image=img)['image']

        return img, age


transform_train = A.Compose([
    A.RandomResizedCrop(
        height=224, 
        width=224, 
        size=(224, 224),   # OBLIGATOIRE sur Kaggle
        scale=(0.85, 1.0)
    ),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.03,
        scale_limit=0.08,
        rotate_limit=10,
        p=0.5
    ),
    A.RandomBrightnessContrast(p=0.35),
    A.CLAHE(p=0.2),
    A.GaussianBlur(blur_limit=(3,5), p=0.2),
    A.CoarseDropout(max_holes=1, max_height=32, max_width=32, p=0.25),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2()
])


transform_test = A.Compose([
    A.Resize(height=224, width=224),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2()
])


# -----------------------------
# Split
# -----------------------------
image_paths = glob.glob("/kaggle/input/utkface-new/UTKFace/*jpg")

train_val_paths, test_paths = train_test_split(image_paths, test_size=0.2, random_state=42)
train_paths, val_paths      = train_test_split(train_val_paths, test_size=0.2, random_state=42)

train_dataset = UTKFaceDataset(train_paths, transform=transform_train)
val_dataset   = UTKFaceDataset(val_paths, transform=transform_test)
test_dataset  = UTKFaceDataset(test_paths, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)


# -----------------------------
# Fonction MixUp légère
# -----------------------------
def mixup(imgs, ages, alpha=0.2):
    if alpha <= 0:
        return imgs, ages

    lam = np.random.beta(alpha, alpha)
    batch_size = imgs.size(0)
    index = torch.randperm(batch_size).to(imgs.device)

    mixed_imgs = lam * imgs + (1 - lam) * imgs[index]
    mixed_ages = lam * ages + (1 - lam) * ages[index]

    return mixed_imgs, mixed_ages


# -----------------------------
# Modèle ViT
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('vit_base_patch16_224', pretrained=True)

# Freeze total
for param in model.parameters():
    param.requires_grad = False

# Unfreeze dernier 1/3
for param in model.blocks[8:].parameters():
    param.requires_grad = True

# Tête
in_features = model.head.in_features
model.head = nn.Sequential(
    nn.Dropout(0.4),
    nn.Linear(in_features, 1)
)
model = model.to(device)

# -----------------------------
# Training setup
# -----------------------------
criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=2, verbose=True
)

epochs = 20
best_val_loss = float('inf')
patience = 5
patience_counter = 0

# -----------------------------
# Training Loop
# -----------------------------
for epoch in range(epochs):
    model.train()
    running_loss = 0

    for imgs, ages in tqdm(train_loader):
        imgs = imgs.to(device)
        ages = ages.float().unsqueeze(1).to(device)

        # MixUp (light)
        imgs, ages = mixup(imgs, ages, alpha=0.15)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, ages)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)

    train_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0
    val_mae = 0

    with torch.no_grad():
        for imgs, ages in val_loader:
            imgs = imgs.to(device)
            ages = ages.float().unsqueeze(1).to(device)
            outputs = model(imgs)

            loss = criterion(outputs, ages)
            val_loss += loss.item() * imgs.size(0)
            val_mae += torch.abs(outputs - ages).sum().item()

    val_loss /= len(val_loader.dataset)
    val_mae  /= len(val_loader.dataset)

    print(f"\nEpoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAE: {val_mae:.2f}")

    scheduler.step(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "/kaggle/working/best_vit_utkface.pth")
        print("✔️ Nouveau meilleur modèle sauvegardé")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⛔ Early stopping")
            break


/tmp/ipykernel_116/3924852830.py:41: UserWarning: Argument(s) 'height, width' are not valid for transform RandomResizedCrop
  A.RandomResizedCrop(
/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_116/3924852830.py:57: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=1, max_height=32, max_width=32, p=0.25),


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
100%|██████████| 475/475 [04:40<00:00,  1.70it/s]



Epoch 1/20
Train Loss: 8.0544 | Val Loss: 6.2705 | Val MAE: 6.27
✔️ Nouveau meilleur modèle sauvegardé


100%|██████████| 475/475 [04:54<00:00,  1.61it/s]


...


Epoch 18/20
Train Loss: 3.7580 | Val Loss: 4.8510 | Val MAE: 4.85


100%|██████████| 475/475 [04:54<00:00,  1.61it/s]



Epoch 19/20
Train Loss: 3.7012 | Val Loss: 4.8065 | Val MAE: 4.81
✔️ Nouveau meilleur modèle sauvegardé


100%|██████████| 475/475 [04:54<00:00,  1.61it/s]



Epoch 20/20
Train Loss: 3.6685 | Val Loss: 4.8328 | Val MAE: 4.83


In [8]:
# -----------------------------
# Imports
# -----------------------------
import os, glob
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
import timm
from tqdm import tqdm

# -----------------------------
# Dataset
# -----------------------------
class UTKFaceDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")
        age = int(os.path.basename(img_path).split('_')[0])
        img = np.array(img)
        if self.transform:
            img = self.transform(image=img)['image']
        return img, age

# -----------------------------
# Augmentations
# -----------------------------
transform_train = A.Compose([
    A.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.CoarseDropout(max_holes=1, max_height=32, max_width=32, p=0.3),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2()
])


transform_test = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2()
])

# -----------------------------
# Chargement des images avec train/val/test split
# -----------------------------
image_paths = glob.glob("/kaggle/input/utkface-new/UTKFace/*jpg")

# 20% test
train_val_paths, test_paths = train_test_split(image_paths, test_size=0.2, random_state=42)

# 80% train, 20% validation (sur le train_val)
train_paths, val_paths = train_test_split(train_val_paths, test_size=0.2, random_state=42)

# Création des datasets
train_dataset = UTKFaceDataset(train_paths, transform=transform_train)
val_dataset   = UTKFaceDataset(val_paths, transform=transform_test)
test_dataset  = UTKFaceDataset(test_paths, transform=transform_test)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# -----------------------------
# Modèle ViT
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('vit_base_patch16_224', pretrained=True)

# Freeze partiel pour éviter l'overfitting
for param in model.parameters():
    param.requires_grad = False

for param in model.blocks[8:].parameters():  # dégel des derniers blocs
    param.requires_grad = True
for param in model.head.parameters():
    param.requires_grad = True

# Tête avec Dropout
in_features = model.head.in_features
model.head = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(in_features, 1)
)
model = model.to(device)

# -----------------------------
# Perte, optimiseur et scheduler
# -----------------------------
criterion = nn.L1Loss()  # moins sensible aux outliers
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)



/tmp/ipykernel_116/2196365205.py:44: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=1, max_height=32, max_width=32, p=0.3),


In [9]:
# -----------------------------
# Entraînement avec early stopping
# -----------------------------
epochs = 20
best_val_loss = float('inf')
patience = 5  # patience plus longue pour éviter arrêt prématuré
counter = 0

for epoch in range(epochs):
    # Training
    model.train()
    running_loss = 0
    for imgs, ages in tqdm(train_loader):
        imgs, ages = imgs.to(device), ages.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, ages)
        loss.backward()
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    train_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0
    val_mae = 0
    with torch.no_grad():
        for imgs, ages in val_loader:
            imgs, ages = imgs.to(device), ages.float().unsqueeze(1).to(device)
            outputs = model(imgs)
            loss = criterion(outputs, ages)
            val_loss += loss.item() * imgs.size(0)
            val_mae += torch.abs(outputs - ages).sum().item()
    val_loss /= len(test_loader.dataset)
    val_mae /= len(test_loader.dataset)

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - Val MAE: {val_mae:.4f}")

    # Scheduler
    scheduler.step(val_loss)

    # Early stopping + sauvegarde
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), "/kaggle/working/best_vit_utkface.pth")
        print("✅ Nouveau meilleur modèle sauvegardé")
    else:
        counter += 1
        if counter >= patience:
            print("⏹ Early stopping déclenché")
            break


100%|██████████| 475/475 [04:54<00:00,  1.61it/s]


Epoch 1/20 - Train Loss: 8.4301 - Val Loss: 4.9656 - Val MAE: 4.9656
✅ Nouveau meilleur modèle sauvegardé


100%|██████████| 475/475 [04:54<00:00,  1.61it/s]


Epoch 2/20 - Train Loss: 6.1679 - Val Loss: 4.3972 - Val MAE: 4.3972
✅ Nouveau meilleur modèle sauvegardé


100%|██████████| 475/475 [04:54<00:00,  1.62it/s]


...

Epoch 18/20 - Train Loss: 2.9383 - Val Loss: 3.8875 - Val MAE: 3.8875


100%|██████████| 475/475 [04:53<00:00,  1.62it/s]


Epoch 19/20 - Train Loss: 2.9159 - Val Loss: 3.8478 - Val MAE: 3.8478


100%|██████████| 475/475 [04:54<00:00,  1.62it/s]


Epoch 20/20 - Train Loss: 2.7337 - Val Loss: 3.8012 - Val MAE: 3.8012
✅ Nouveau meilleur modèle sauvegardé


In [10]:

# -----------------------------
# 8️⃣ Évaluation MAE
# -----------------------------
model.load_state_dict(torch.load("/kaggle/working/best_vit_utkface.pth"))
model.eval()

ages_true = []
ages_pred = []

with torch.no_grad():
    for imgs, ages in test_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        ages_pred.extend(outputs.cpu().numpy())
        ages_true.extend(ages.numpy())

ages_pred = np.array(ages_pred).flatten()
ages_true = np.array(ages_true).flatten()
mae = np.mean(np.abs(ages_true - ages_pred))
print(f"📊 Mean Absolute Error (MAE) sur le test set : {mae:.2f} ans")


📊 Mean Absolute Error (MAE) sur le test set : 4.60 ans


In [11]:
# Sauvegarder uniquement les poids du modèle
torch.save(model.state_dict(), "/kaggle/working/vit_utkface.pth")

# Ou sauvegarder le modèle complet (architecture + poids)
torch.save(model, "/kaggle/working/vit_utkface_full.pth")
